# Module Five Assignment: Cartpole Problem
Review the code in this notebook and in the score_logger.py file in the *scores* folder (directory). Once you have reviewed the code, return to this notebook and select **Cell** and then **Run All** from the menu bar to run this code. The code takes several minutes to run.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings (only show errors)

import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from scores.score_logger import ScoreLogger

ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995

class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(learning_rate=LEARNING_RATE))  # Use learning_rate instead of lr

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

    def save_model(self, path="dqn_cartpole_model.h5"):
        self.model.save(path)


def cartpole():
    env = gym.make(ENV_NAME)
    score_logger = ScoreLogger(ENV_NAME)
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    scores = deque(maxlen=100)  # To track scores for calculating the average

    while True:
        run += 1
        state, _ = env.reset()  # Unpack the tuple
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            # env.render()
            action = dqn_solver.act(state)
            next_state_info = env.step(action)
            if len(next_state_info) == 5:
                state_next, reward, terminal, truncated, info = next_state_info
                terminal = terminal or truncated  # Combine terminal and truncated states
            else:
                state_next, reward, terminal, info = next_state_info
            reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                scores.append(step)
                avg_score = np.mean(scores)
                print(f"Run: {run}, exploration: {dqn_solver.exploration_rate}, score: {step}")
                score_logger.add_score(step, run)
                if avg_score >= 195.0:
                    print(f"Solved in {run} runs, {run} total runs.")
                    dqn_solver.save_model()
                    return
                break
            dqn_solver.experience_replay()

cartpole()


/Users/victor/Desktop/Projects/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1/1 [==============================] - 0s 10ms/step


/Users/victor/Desktop/Projects/myenv/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


1/1 [==============================] - 0s 9ms/step
Run: 1, exploration: 0.8955869907338783, score: 42
Scores: (min: 42, avg: 42, max: 42)

1/1 [==============================] - 0s 9ms/step
Run: 2, exploration: 0.8183201210226743, score: 19
Scores: (min: 19, avg: 30.5, max: 42)

1/1 [==============================] - 0s 9ms/step
Run: 3, exploration: 0.7552531090661897, score: 17
Scores: (min: 17, avg: 26, max: 42)

1/1 [==============================] - 0s 9ms/step
Run: 4, exploration: 0.6730128848950395, score: 24
Scores: (min: 17, avg: 25.5, max: 42)

1/1 [==============================] - 0s 9ms/step
Run: 5, exploration: 0.6305556603555866, score: 14
Scores: (min: 14, avg: 23.2, max: 42)

1/1 [==============================] - 0s 9ms/step
Run: 6, exploration: 0.5761543988830038, score: 19
Scores: (min: 14, avg: 22.5, max: 42)

1/1 [==============================] - 0s 10ms/step
Run: 7, exploration: 0.5371084840724134, score: 15
Scores: (min: 14, avg: 21.428571428571427, max: 42)

1/

Note: If the code is running properly, you should begin to see output appearing above this code block. It will take several minutes, so it is recommended that you let this code run in the background while completing other work. When the code has finished, it will print output saying, "Solved in _ runs, _ total runs."

You may see an error about not having an exit command. This error does not affect the program's functionality and results from the steps taken to convert the code from Python 2.x to Python 3. Please disregard this error.